In [1]:
import pandas as pd
import numpy as np
import re
import random
import time

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
full_name_df=pd.read_csv("/content/gdrive/MyDrive/Data /ML datasets/processed-full-name-dup.csv")
full_name_no_dup_df=pd.read_csv("/content/gdrive/MyDrive/Data /ML datasets/processed-full-name.csv")
first_name_df = pd.read_csv("/content/gdrive/MyDrive/Data /ML datasets/processed-first-name.csv")
last_name_df = pd.read_csv("/content/gdrive/MyDrive/Data /ML datasets/processed-last-name.csv")
middle_name_df = pd.read_csv("/content/gdrive/MyDrive/Data /ML datasets/processed-middle-name.csv")
middle_first_name_df = pd.read_csv("/content/gdrive/MyDrive/Data /ML datasets/processed-middle-first-name.csv")

In [5]:
def lowerize(text):
  patterns = {
  '[àáảãạăắằẵặẳâầấậẫẩ]': 'a',
  '[đ]': 'd',
  '[èéẻẽẹêềếểễệ]': 'e',
  '[ìíỉĩị]': 'i',
  '[òóỏõọôồốổỗộơờớởỡợ]': 'o',
  '[ùúủũụưừứửữự]': 'u',
  '[ỳýỷỹỵ]': 'y'
  }
  output = text
  for regex, replace in patterns.items():
    output = re.sub(regex, replace, output)
    # deal with upper case
    output = re.sub(regex.upper(), replace.upper(), output)
  return output.lower()

In [10]:
class TF_IDF():
  def __init__(self, corpus, dictionary=None, max_count=None, min_count=None, normalize_tf=False, smooth=True, normalize_tfidf=None):
    self.corpus=corpus
    self.max_count=max_count
    self.min_count=min_count
    self.normalize_tf=normalize_tf
    self.smooth=smooth
    self.normalize_tfidf=normalize_tfidf
    self.dictionary = dictionary if dictionary!=None else self.create_dictionary()
    self.num_word=len(self.dictionary)
    self.word_to_index = self.map_word_to_index()
    self.num_document = len(self.corpus)
    self.matrix_word_count = self.create_count_matrix()
  #return the word in dictionary given index
  def retrieve_word(self, index):
    return self.dictionary[index]
  def create_dictionary(self):
    if self.max_count==None and self.min_count==None:
      set_word = set()
      for doc in self.corpus:
        set_word = set_word.union(set(self.word_extraction(doc.lower())))
    else:
      set_word = set()
      map_word_count = self.map_word_to_count()
      for doc in self.corpus:
        list_word=self.word_extraction(doc.lower())
        for word in list_word:
          if self.min_count!=None:
            if map_word_count[word] < self.min_count:
              continue
          if self.max_count!=None:
            if map_word_count[word] > self.max_count:
              continue
          set_word.add(word)
    return sorted(list(set_word))
  def retrieve_index(self, word):
    return self.word_to_index[word.lower()]

  def word_extraction(self, document):
    split_word=document.split()
    return split_word

  def map_word_to_count(self):
    dict_word_count = dict()
    for i in range(len(self.corpus)):
      list_word = self.word_extraction(self.corpus[i].lower())
      for j in range(len(list_word)):
        dict_word_count[list_word[j]] = dict_word_count.get(list_word[j],0)+1
    return dict_word_count

  def map_word_to_index(self):
    dict_encode=dict()
    for i in range(len(self.dictionary)):
      dict_encode[self.dictionary[i]]=i
    return dict_encode

  def create_count_matrix(self):
    mat = np.zeros((self.num_document, self.num_word))
    for i in range(len(self.corpus)):
      document = self.corpus[i].lower()
      list_word = self.word_extraction(document)
      for j in range(len(list_word)):
        ind = self.retrieve_index(list_word[j])
        mat[i, ind]+=1
    return mat

  def compute_tf(self):
    length_name = np.sum(self.matrix_word_count, axis=1)
    if self.normalize_tf==True:
      return self.matrix_word_count/np.reshape(length_name, (-1,1))
    else:
      return self.matrix_word_count

  def compute_idf(self):
    tmp = np.copy(self.matrix_word_count)
    tmp[tmp!=0]=1
    num_doc_having_word = np.sum(tmp, axis=0)
    if self.smooth == True:
      num_doc_having_word = np.log((self.num_document+1) / (num_doc_having_word+1)) + 1
    else:
      num_doc_having_word = np.log(self.num_document / num_doc_having_word) + 1

    return np.reshape(num_doc_having_word, (1, self.num_word))
  def compute_tf_idf(self):
    tf = self.compute_tf()
    idf = self.compute_idf()
    tfidf = tf * idf
    if self.normalize_tfidf==None:
      return tfidf
    elif self.normalize_tfidf == "l2":
      sum_squares = np.reshape(np.diag(tfidf.dot(tfidf)), (1, -1))
      return tfidf / sum_squares
    elif self.normalize_tfidf == "l1":
      sum_row = np.reshape(np.sum(tfidf, axis=1), (1, -1))
      return tfidf / sum_row

In [11]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

In [12]:
model = Sequential()
model.add(Dense(100, input_dim=100, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

In [13]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


Test with fullname dataset with duplicates and truncated

In [14]:
full_name_data = full_name_df["Full_Name"].tolist()
full_name_label = full_name_df["Gender"].to_numpy()

In [15]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
tfidf_full_name_data=TF_IDF(full_name_data).compute_tf_idf()
svd = TruncatedSVD(n_components=100)
normalizer = Normalizer(copy=False)
full_name_normalized = normalizer.fit_transform(tfidf_full_name_data)
full_name_svd = svd.fit_transform(tfidf_full_name_data)
full_name_svd.shape

(26373, 100)

In [16]:
fullname_train_X, fullname_test_X, fullname_train_y, fullname_test_y = train_test_split(full_name_svd, full_name_label.reshape(-1,1), test_size=0.2, random_state=42)

In [17]:
start_time = time.time()
model.fit(fullname_train_X,fullname_train_y,epochs=150,validation_split=0.2, batch_size=200)
print("Time: "+str(time.time()-start_time))

Epoch 1/150
85/85 [==============================] - 2s 9ms/step - loss: 2.1929 - accuracy: 0.6548 - val_loss: 1.0146 - val_accuracy: 0.8443
Epoch 2/150
85/85 [==============================] - 1s 6ms/step - loss: 0.7276 - accuracy: 0.8887 - val_loss: 0.5927 - val_accuracy: 0.9273
Epoch 3/150
85/85 [==============================] - 1s 6ms/step - loss: 0.5163 - accuracy: 0.9117 - val_loss: 0.4550 - val_accuracy: 0.9187
Epoch 4/150
85/85 [==============================] - 1s 6ms/step - loss: 0.4439 - accuracy: 0.9181 - val_loss: 0.4220 - val_accuracy: 0.9147
Epoch 5/150
85/85 [==============================] - 1s 10ms/step - loss: 0.4159 - accuracy: 0.9197 - val_loss: 0.4063 - val_accuracy: 0.9315
Epoch 6/150
85/85 [==============================] - 1s 9ms/step - loss: 0.4034 - accuracy: 0.9192 - val_loss: 0.3848 - val_accuracy: 0.9303
Epoch 7/150
85/85 [==============================] - 1s 9ms/step - loss: 0.3928 - accuracy: 0.9191 - val_loss: 0.3801 - val_accuracy: 0.9339
Epoch 8/150


In [18]:
loss, accuracy = model.evaluate(fullname_test_X, fullname_test_y)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

165/165 [==============================] - 0s 2ms/step - loss: 0.2716 - accuracy: 0.9310
Test Loss: 0.2716262936592102
Test Accuracy: 0.9309952855110168


Test with fullname dataset with no truncated svd

In [26]:
raw_fullname_train_X, raw_fullname_test_X, raw_fullname_train_y, raw_fullname_test_y = train_test_split(tfidf_full_name_data, full_name_label, test_size=0.2, random_state=42)

In [ ]:
model1 = Sequential()
model1.add(Dense(100, input_dim=tfidf_full_name_data.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model1.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model1.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model1.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model1.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model1.add(Dense(1, activation='sigmoid'))

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
model1.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
start_time = time.time()
model1.fit(raw_fullname_train_X,raw_fullname_train_y,epochs=150,validation_split=0.2, batch_size=200)
print("Time: "+str(time.time()-start_time))

Epoch 1/150
85/85 [==============================] - 2s 13ms/step - loss: 2.1483 - accuracy: 0.5781 - val_loss: 0.9294 - val_accuracy: 0.5656
Epoch 2/150
85/85 [==============================] - 1s 13ms/step - loss: 0.6174 - accuracy: 0.8855 - val_loss: 0.4537 - val_accuracy: 0.9502
Epoch 3/150
85/85 [==============================] - 1s 15ms/step - loss: 0.4212 - accuracy: 0.9443 - val_loss: 0.3872 - val_accuracy: 0.9502
Epoch 4/150
85/85 [==============================] - 1s 17ms/step - loss: 0.3843 - accuracy: 0.9447 - val_loss: 0.3714 - val_accuracy: 0.9498
Epoch 5/150
85/85 [==============================] - 1s 16ms/step - loss: 0.3650 - accuracy: 0.9462 - val_loss: 0.3713 - val_accuracy: 0.9491
Epoch 6/150
85/85 [==============================] - 1s 11ms/step - loss: 0.3544 - accuracy: 0.9450 - val_loss: 0.3822 - val_accuracy: 0.9360
Epoch 7/150
85/85 [==============================] - 1s 11ms/step - loss: 0.3455 - accuracy: 0.9470 - val_loss: 0.3361 - val_accuracy: 0.9498
Epoch 

In [ ]:
loss, accuracy = model1.evaluate(raw_fullname_test_X, raw_fullname_test_y)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

165/165 [==============================] - 0s 2ms/step - loss: 0.2305 - accuracy: 0.9560
Test Loss: 0.23045121133327484
Test Accuracy: 0.9560189843177795


Test with dataset with drop-duplicate and truncated_svd

In [28]:
fullname_no_dup_data = full_name_no_dup_df["Full_Name"].tolist()
fullname_no_dup_label = full_name_no_dup_df["Gender"].to_numpy()

In [29]:
from sklearn.decomposition import TruncatedSVD
tfidf_fullname_no_dup_data=TF_IDF(fullname_no_dup_data).compute_tf_idf()
svd = TruncatedSVD(n_components=100)
full_name_no_dup_svd = svd.fit_transform(tfidf_fullname_no_dup_data)

In [ ]:
fn_no_dup_train_X, fn_no_dup_test_X, fn_no_dup_train_y, fn_no_dup_test_y = train_test_split(full_name_no_dup_svd, fullname_no_dup_label, test_size=0.2, random_state=42)

In [ ]:
model2 = Sequential()
model2.add(Dense(100, input_dim=full_name_no_dup_svd.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model2.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model2.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model2.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model2.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model2.add(Dense(1, activation='sigmoid'))

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
model2.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
start_time=time.time()
model2.fit(fn_no_dup_train_X,fn_no_dup_train_y,epochs=150,validation_split=0.2, batch_size=200)
print("Time: "+str(time.time()-start_time))

Epoch 1/150
67/67 [==============================] - 2s 11ms/step - loss: 2.5021 - accuracy: 0.8579 - val_loss: 1.4328 - val_accuracy: 0.9101
Epoch 2/150
67/67 [==============================] - 0s 7ms/step - loss: 1.0100 - accuracy: 0.9179 - val_loss: 0.7176 - val_accuracy: 0.9173
Epoch 3/150
67/67 [==============================] - 0s 6ms/step - loss: 0.5966 - accuracy: 0.9242 - val_loss: 0.5268 - val_accuracy: 0.9176
Epoch 4/150
67/67 [==============================] - 0s 7ms/step - loss: 0.4584 - accuracy: 0.9283 - val_loss: 0.4352 - val_accuracy: 0.9149
Epoch 5/150
67/67 [==============================] - 0s 6ms/step - loss: 0.3972 - accuracy: 0.9290 - val_loss: 0.4050 - val_accuracy: 0.9200
Epoch 6/150
67/67 [==============================] - 1s 10ms/step - loss: 0.3633 - accuracy: 0.9306 - val_loss: 0.3982 - val_accuracy: 0.9116
Epoch 7/150
67/67 [==============================] - 1s 10ms/step - loss: 0.3426 - accuracy: 0.9321 - val_loss: 0.3502 - val_accuracy: 0.9227
Epoch 8/15

In [ ]:
loss, accuracy = model2.evaluate(fn_no_dup_test_X, fn_no_dup_test_y)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

130/130 [==============================] - 0s 2ms/step - loss: 0.2641 - accuracy: 0.9305
Test Loss: 0.2641112208366394
Test Accuracy: 0.9304515719413757


Test with drop duplicate dataset without truncated

In [ ]:
raw_no_dup_train_X, raw_no_dup_test_X, raw_no_dup_train_y, raw_no_dup_test_y = train_test_split(tfidf_fullname_no_dup_data, fullname_no_dup_label, test_size=0.2, random_state=42)

In [ ]:
model3 = Sequential()
model3.add(Dense(100, input_dim=tfidf_fullname_no_dup_data.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dense(1, activation='sigmoid'))

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
model3.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
start_time=time.time()
model3.fit(raw_no_dup_train_X,raw_no_dup_train_y,epochs=150,validation_split=0.2, batch_size=200)
print("Time: "+str(time.time()-start_time))

Epoch 1/150
67/67 [==============================] - 2s 13ms/step - loss: 2.3690 - accuracy: 0.8370 - val_loss: 1.0468 - val_accuracy: 0.9107
Epoch 2/150
67/67 [==============================] - 1s 10ms/step - loss: 0.7157 - accuracy: 0.9360 - val_loss: 0.5153 - val_accuracy: 0.9372
Epoch 3/150
67/67 [==============================] - 1s 10ms/step - loss: 0.4260 - accuracy: 0.9435 - val_loss: 0.3803 - val_accuracy: 0.9436
Epoch 4/150
67/67 [==============================] - 1s 10ms/step - loss: 0.3507 - accuracy: 0.9454 - val_loss: 0.3469 - val_accuracy: 0.9360
Epoch 5/150
67/67 [==============================] - 1s 10ms/step - loss: 0.3213 - accuracy: 0.9431 - val_loss: 0.3185 - val_accuracy: 0.9375
Epoch 6/150
67/67 [==============================] - 1s 10ms/step - loss: 0.3021 - accuracy: 0.9460 - val_loss: 0.2969 - val_accuracy: 0.9442
Epoch 7/150
67/67 [==============================] - 1s 10ms/step - loss: 0.2895 - accuracy: 0.9469 - val_loss: 0.2866 - val_accuracy: 0.9445
Epoch 

In [ ]:
loss, accuracy = model3.evaluate(raw_no_dup_test_X, raw_no_dup_test_y)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

130/130 [==============================] - 0s 2ms/step - loss: 0.2140 - accuracy: 0.9413
Test Loss: 0.21395626664161682
Test Accuracy: 0.9413185119628906


Test some metrics

In [9]:

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import log_loss, precision_recall_curve, PrecisionRecallDisplay, roc_curve, RocCurveDisplay, roc_auc_score

class Metric:
    def __init__(self, y_true:np.ndarray, y_pred:np.ndarray, y_proba:np.ndarray=None):
        self.y_true = y_true
        self.y_pred = y_pred
        self.y_proba = y_proba

    # Accuracy
    def getAccuracy(self):
        return accuracy_score(self.y_true, self.y_pred)

    # Recall
    def getRecallClass0(self):
        return recall_score(self.y_true, self.y_pred, pos_label=0, average="binary")
    def getRecallClass1(self):
        return recall_score(self.y_true, self.y_pred, pos_label=1, average="binary")
    def getMacroAvgRecall(self):
        return recall_score(self.y_true, self.y_pred, average="macro")
    def getWeightedAvgRecall(self):
        return recall_score(self.y_true, self.y_pred, average="weighted")

    # Precision
    def getPrecisionClass0(self):
        return precision_score(self.y_true, self.y_pred, pos_label=0, average="binary")
    def getPrecisionClass1(self):
        return precision_score(self.y_true, self.y_pred, pos_label=1, average="binary")
    def getMacroAvgPrecision(self):
        return precision_score(self.y_true, self.y_pred, average="macro")
    def getWeightedAvgPrecision(self):
        return precision_score(self.y_true, self.y_pred, average="weighted")

    # F1
    def getF1Class0(self):
        return f1_score(self.y_true, self.y_pred, pos_label=0, average="binary")
    def getF1Class1(self):
        return f1_score(self.y_true, self.y_pred, pos_label=1, average="binary")
    def getMacroAvgF1(self):
        return f1_score(self.y_true, self.y_pred, average="macro")
    def getWeightedAvgF1(self):
        return f1_score(self.y_true, self.y_pred, average="weighted")

    # Confusion matrix
    def getConfusionMatrix(self):
        cm = confusion_matrix(self.y_true, self.y_pred)
        cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
        return cm, cm_disp

    # Classification report
    def getClassificationReport(self):
        return classification_report(self.y_true, self.y_pred)

    ##########

    # Log loss
    def getLogLoss(self):
        return log_loss(self.y_true, self.y_proba)

    # Precision-recall curve
    def getPrecisionRecallCurveClass0(self):
        prec, recall, _ = precision_recall_curve(self.y_true, 1 - self.y_proba, pos_label=0)
        pr_disp = PrecisionRecallDisplay(prec, recall)
        return pr_disp

    def getPrecisionRecallCurveClass1(self):
        prec, recall, _ = precision_recall_curve(self.y_true, self.y_proba, pos_label=1)
        pr_disp = PrecisionRecallDisplay(prec, recall)
        return pr_disp

    # ROC curve
    def getRocCurveClass0(self):
        fpr, tpr, _ = roc_curve(self.y_true, 1 - self.y_proba, pos_label=0)
        roc_disp = RocCurveDisplay(fpr=fpr, tpr=tpr)
        return roc_disp

    def getRocCurveClass1(self):
        fpr, tpr, _ = roc_curve(self.y_true, self.y_proba, pos_label=1)
        roc_disp = RocCurveDisplay(fpr=fpr, tpr=tpr)
        return roc_disp

    def getRocAucScore(self):
        return roc_auc_score(self.y_true, self.y_proba)

Test on fullname dataset

In [19]:
fullname_y_pred_prob = model.predict(fullname_test_X)
fullname_y_pred = (fullname_y_pred_prob > 0.5).astype(int)
fullname_metric = Metric(fullname_test_y, fullname_y_pred, fullname_y_pred_prob)

165/165 [==============================] - 0s 1ms/step


In [20]:
print(fullname_metric.getPrecisionClass0())
print(fullname_metric.getRecallClass0())
print(fullname_metric.getF1Class0())

0.9664328657314629
0.866576819407008
0.9137849360492657


In [21]:
print(fullname_metric.getPrecisionClass1())
print(fullname_metric.getRecallClass1())
print(fullname_metric.getF1Class1())

0.909423604757548
0.9780255821580847
0.9424778761061946


In [22]:
print(fullname_metric.getAccuracy())
print(fullname_metric.getLogLoss())
print(fullname_metric.getRocAucScore())

0.9309952606635071
0.17900650175697372
0.9798640769203343


Test on first name dataset

In [24]:
first_name_data = first_name_df["Full_Name"].tolist()
first_name_label = first_name_df["Gender"].to_numpy()

In [31]:
first_name_X = TF_IDF(first_name_data).compute_tf_idf()
svd = TruncatedSVD(n_components=100)
first_name_svd = svd.fit_transform(first_name_X)

In [37]:
firstname_train_X, firstname_test_X, firstname_train_y, firstname_test_y = train_test_split(first_name_svd, first_name_label, test_size=0.2, random_state=42)

In [38]:
firstname_model = Sequential()
firstname_model.add(Dense(100, input_dim=firstname_train_X.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
firstname_model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
firstname_model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
firstname_model.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
firstname_model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
firstname_model.add(Dense(1, activation='sigmoid'))

In [40]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
firstname_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [41]:
firstname_model.fit(firstname_train_X,firstname_train_y,epochs=150,validation_split=0.2, batch_size=200)

Epoch 1/150
86/86 [==============================] - 2s 8ms/step - loss: 2.2204 - accuracy: 0.7617 - val_loss: 1.0943 - val_accuracy: 0.8226
Epoch 2/150
86/86 [==============================] - 1s 6ms/step - loss: 0.7819 - accuracy: 0.8247 - val_loss: 0.6180 - val_accuracy: 0.8264
Epoch 3/150
86/86 [==============================] - 0s 6ms/step - loss: 0.5781 - accuracy: 0.8249 - val_loss: 0.5347 - val_accuracy: 0.8312
Epoch 4/150
86/86 [==============================] - 0s 6ms/step - loss: 0.5323 - accuracy: 0.8281 - val_loss: 0.5272 - val_accuracy: 0.8205
Epoch 5/150
86/86 [==============================] - 1s 6ms/step - loss: 0.5136 - accuracy: 0.8245 - val_loss: 0.4931 - val_accuracy: 0.8361
Epoch 6/150
86/86 [==============================] - 1s 6ms/step - loss: 0.5016 - accuracy: 0.8274 - val_loss: 0.4889 - val_accuracy: 0.8329
Epoch 7/150
86/86 [==============================] - 1s 6ms/step - loss: 0.4937 - accuracy: 0.8291 - val_loss: 0.4805 - val_accuracy: 0.8338
Epoch 8/150
8

In [42]:
firstname_y_pred_prob = firstname_model.predict(firstname_test_X)
firstname_y_pred = (firstname_y_pred_prob>0.5).astype(int)
firstname_metric = Metric(firstname_test_y, firstname_y_pred, firstname_y_pred_prob)

168/168 [==============================] - 0s 1ms/step


In [43]:
print(firstname_metric.getPrecisionClass0())
print(firstname_metric.getRecallClass0())
print(firstname_metric.getF1Class0())

0.8822571893651655
0.7265415549597856
0.7968635138446459


In [44]:
print(firstname_metric.getPrecisionClass1())
print(firstname_metric.getRecallClass1())
print(firstname_metric.getF1Class1())

0.8264814289764673
0.9307151979565773
0.8755068328577865


In [45]:
print(firstname_metric.getAccuracy())
print(firstname_metric.getLogLoss())
print(firstname_metric.getRocAucScore())

0.8456238361266294
0.350798346820495
0.9189351866118376


Test with middle name dataset

In [49]:
middle_name_data = middle_name_df["Full_Name"].tolist()
middle_name_label = middle_name_df["Gender"].to_numpy()

In [47]:
middle_name_X = TF_IDF(middle_name_data).compute_tf_idf()
svd = TruncatedSVD(n_components=100)
middle_name_svd = svd.fit_transform(middle_name_X)

In [51]:
middle_train_X, middle_test_X, middle_train_y, middle_test_y = train_test_split(middle_name_svd, middle_name_label, test_size=0.2, random_state=42)

In [55]:
middle_model = Sequential()
middle_model.add(Dense(100, input_dim=firstname_train_X.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
middle_model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
middle_model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
middle_model.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
middle_model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
middle_model.add(Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
middle_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [56]:
middle_model.fit(middle_train_X,middle_train_y,epochs=150,validation_split=0.2, batch_size=200)

Epoch 1/150
85/85 [==============================] - 2s 13ms/step - loss: 2.1803 - accuracy: 0.8582 - val_loss: 1.0467 - val_accuracy: 0.8922
Epoch 2/150
85/85 [==============================] - 1s 10ms/step - loss: 0.6959 - accuracy: 0.9096 - val_loss: 0.5347 - val_accuracy: 0.8988
Epoch 3/150
85/85 [==============================] - 1s 14ms/step - loss: 0.4447 - accuracy: 0.9116 - val_loss: 0.4318 - val_accuracy: 0.8946
Epoch 4/150
85/85 [==============================] - 1s 11ms/step - loss: 0.3822 - accuracy: 0.9115 - val_loss: 0.3933 - val_accuracy: 0.9003
Epoch 5/150
85/85 [==============================] - 1s 7ms/step - loss: 0.3556 - accuracy: 0.9092 - val_loss: 0.3723 - val_accuracy: 0.9022
Epoch 6/150
85/85 [==============================] - 1s 6ms/step - loss: 0.3427 - accuracy: 0.9106 - val_loss: 0.3602 - val_accuracy: 0.9031
Epoch 7/150
85/85 [==============================] - 1s 6ms/step - loss: 0.3337 - accuracy: 0.9114 - val_loss: 0.3538 - val_accuracy: 0.9038
Epoch 8/1

In [57]:
middlename_y_pred_prob = middle_model.predict(middle_test_X)
middlename_y_pred = (middlename_y_pred_prob>0.5).astype(int)
middlename_metric = Metric(middle_test_y, middlename_y_pred, middlename_y_pred_prob)

166/166 [==============================] - 1s 4ms/step


In [58]:
print(middlename_metric.getPrecisionClass0())
print(middlename_metric.getRecallClass0())
print(middlename_metric.getF1Class0())

0.9558521560574949
0.8148796498905908
0.8797543113630995


In [59]:
print(middlename_metric.getPrecisionClass1())
print(middlename_metric.getRecallClass1())
print(middlename_metric.getF1Class1())

0.8733912002394493
0.9713715046604527
0.919779353821907


In [60]:
print(middlename_metric.getAccuracy())
print(middlename_metric.getLogLoss())
print(middlename_metric.getRocAucScore())

0.903762525997353
0.23518215810629775
0.9610463218990288


Test on last name dataset

In [61]:
last_name_data = last_name_df["Full_Name"].tolist()
last_name_label = last_name_df["Gender"].to_numpy()

In [62]:
last_name_X = TF_IDF(last_name_data).compute_tf_idf()
svd = TruncatedSVD(n_components=100)
last_name_svd = svd.fit_transform(last_name_X)

In [63]:
last_train_X, last_test_X, last_train_y, last_test_y = train_test_split(last_name_svd, last_name_label, test_size=0.2, random_state=42)

In [68]:
lastname_model = Sequential()
lastname_model.add(Dense(100, input_dim=firstname_train_X.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
lastname_model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
lastname_model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
lastname_model.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
lastname_model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
lastname_model.add(Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
lastname_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [69]:
lastname_model.fit(last_train_X,last_train_y,epochs=150,validation_split=0.2, batch_size=200)

Epoch 1/150
86/86 [==============================] - 2s 16ms/step - loss: 2.2159 - accuracy: 0.5727 - val_loss: 1.0880 - val_accuracy: 0.5822
Epoch 2/150
86/86 [==============================] - 1s 7ms/step - loss: 0.8354 - accuracy: 0.5759 - val_loss: 0.7269 - val_accuracy: 0.5822
Epoch 3/150
86/86 [==============================] - 1s 9ms/step - loss: 0.7079 - accuracy: 0.5759 - val_loss: 0.6930 - val_accuracy: 0.5822
Epoch 4/150
86/86 [==============================] - 1s 10ms/step - loss: 0.6895 - accuracy: 0.5759 - val_loss: 0.6837 - val_accuracy: 0.5822
Epoch 5/150
86/86 [==============================] - 1s 10ms/step - loss: 0.6836 - accuracy: 0.5759 - val_loss: 0.6803 - val_accuracy: 0.5822
Epoch 6/150
86/86 [==============================] - 1s 10ms/step - loss: 0.6819 - accuracy: 0.5759 - val_loss: 0.6797 - val_accuracy: 0.5822
Epoch 7/150
86/86 [==============================] - 1s 9ms/step - loss: 0.6817 - accuracy: 0.5759 - val_loss: 0.6798 - val_accuracy: 0.5822
Epoch 8/1

In [76]:
lastname_y_pred_prob = lastname_model.predict(last_test_X)
lastname_y_pred = (lastname_y_pred_prob>0.5).astype(int)
lastname_metric = Metric(last_test_y, lastname_y_pred, lastname_y_pred_prob)


168/168 [==============================] - 0s 1ms/step


In [73]:
print(lastname_metric.getPrecisionClass0())
print(lastname_metric.getRecallClass0())
print(lastname_metric.getF1Class0())

0.0
0.0
0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [74]:
print(lastname_metric.getPrecisionClass1())
print(lastname_metric.getRecallClass1())
print(lastname_metric.getF1Class1())

0.5767225325884544
1.0
0.7315460021259006


In [75]:
print(lastname_metric.getAccuracy())
print(lastname_metric.getLogLoss())
print(lastname_metric.getRocAucScore())

0.5767225325884544
0.681336264571845
0.5


Test on middle first name dataset


In [77]:
middle_first_name_data = middle_first_name_df["Full_Name"].tolist()
middle_first_name_label = middle_first_name_df["Gender"].to_numpy()

In [78]:
middle_first_name_X = TF_IDF(middle_first_name_data).compute_tf_idf()
svd = TruncatedSVD(n_components=100)
middle_first_name_svd = svd.fit_transform(middle_first_name_X)

In [84]:
midfirst_train_X, midfirst_test_X, midfirst_train_y, midfirst_test_y = train_test_split(middle_first_name_svd, middle_first_name_label, test_size=0.2, random_state=42)

In [85]:
midfirstname_model = Sequential()
midfirstname_model.add(Dense(100, input_dim=firstname_train_X.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
midfirstname_model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
midfirstname_model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
midfirstname_model.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
midfirstname_model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
midfirstname_model.add(Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
midfirstname_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [86]:
midfirstname_model.fit(midfirst_train_X, midfirst_train_y,epochs=150,validation_split=0.2, batch_size=200)

Epoch 1/150
86/86 [==============================] - 2s 7ms/step - loss: 2.1393 - accuracy: 0.8962 - val_loss: 1.0073 - val_accuracy: 0.9299
Epoch 2/150
86/86 [==============================] - 0s 5ms/step - loss: 0.6673 - accuracy: 0.9368 - val_loss: 0.4820 - val_accuracy: 0.9390
Epoch 3/150
86/86 [==============================] - 0s 6ms/step - loss: 0.4094 - accuracy: 0.9417 - val_loss: 0.3667 - val_accuracy: 0.9392
Epoch 4/150
86/86 [==============================] - 0s 5ms/step - loss: 0.3401 - accuracy: 0.9415 - val_loss: 0.3461 - val_accuracy: 0.9232
Epoch 5/150
86/86 [==============================] - 0s 5ms/step - loss: 0.3120 - accuracy: 0.9398 - val_loss: 0.3308 - val_accuracy: 0.9320
Epoch 6/150
86/86 [==============================] - 0s 5ms/step - loss: 0.2944 - accuracy: 0.9408 - val_loss: 0.2988 - val_accuracy: 0.9411
Epoch 7/150
86/86 [==============================] - 0s 6ms/step - loss: 0.2843 - accuracy: 0.9409 - val_loss: 0.2912 - val_accuracy: 0.9369
Epoch 8/150
8

In [90]:
midfirstname_y_pred_prob = midfirstname_model.predict(midfirst_test_X)
midfirstname_y_pred = (midfirstname_y_pred_prob>0.5).astype(int)
midfirstname_metric = Metric(midfirst_test_y, midfirstname_y_pred, midfirstname_y_pred_prob)

168/168 [==============================] - 0s 1ms/step


In [91]:
print(midfirstname_metric.getPrecisionClass0())
print(midfirstname_metric.getRecallClass0())
print(midfirstname_metric.getF1Class0())

0.9547785547785548
0.9187976671152983
0.9364426154549612


In [92]:
print(midfirstname_metric.getPrecisionClass1())
print(midfirstname_metric.getRecallClass1())
print(midfirstname_metric.getF1Class1())

0.943875968992248
0.9691181152499204
0.9563305058121269


In [93]:
print(midfirstname_metric.getAccuracy())
print(midfirstname_metric.getLogLoss())
print(midfirstname_metric.getRocAucScore())

0.9482309124767225
0.1441853367701607
0.9846669663257723


Test on raw data set

In [94]:
raw_df = pd.read_csv("/content/gdrive/MyDrive/Data /name_full.csv")

In [95]:
raw_data = raw_df["Full_Name"].tolist()
raw_label = raw_df["Gender"].to_numpy()

In [96]:
raw_X = TF_IDF(raw_data).compute_tf_idf()
svd = TruncatedSVD(n_components=100)
raw_svd = svd.fit_transform(raw_X)

In [97]:
raw_train_X, raw_test_X, raw_train_y, raw_test_y = train_test_split(raw_svd, raw_label, test_size=0.2, random_state=42)

In [98]:
test_model = Sequential()
test_model.add(Dense(100, input_dim=raw_train_X.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.01)))
test_model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
test_model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
test_model.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
test_model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
test_model.add(Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
test_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [103]:
start_time=time.time()
test_model.fit(raw_train_X, raw_train_y, epochs=150, validation_split=0.15, batch_size=1000)
print("Time: "+str(time.time()-start_time))

Epoch 1/150
19/19 [==============================] - 0s 19ms/step - loss: 0.2046 - accuracy: 0.9502 - val_loss: 0.2229 - val_accuracy: 0.9435
Epoch 2/150
19/19 [==============================] - 0s 12ms/step - loss: 0.2041 - accuracy: 0.9498 - val_loss: 0.2369 - val_accuracy: 0.9345
Epoch 3/150
19/19 [==============================] - 0s 11ms/step - loss: 0.2044 - accuracy: 0.9499 - val_loss: 0.2210 - val_accuracy: 0.9448
Epoch 4/150
19/19 [==============================] - 0s 12ms/step - loss: 0.2043 - accuracy: 0.9509 - val_loss: 0.2233 - val_accuracy: 0.9432
Epoch 5/150
19/19 [==============================] - 0s 12ms/step - loss: 0.2052 - accuracy: 0.9491 - val_loss: 0.2354 - val_accuracy: 0.9354
Epoch 6/150
19/19 [==============================] - 0s 12ms/step - loss: 0.2046 - accuracy: 0.9499 - val_loss: 0.2237 - val_accuracy: 0.9413
Epoch 7/150
19/19 [==============================] - 0s 12ms/step - loss: 0.2047 - accuracy: 0.9503 - val_loss: 0.2257 - val_accuracy: 0.9432
Epoch 

In [104]:
loss, accuracy = test_model.evaluate(raw_test_X, raw_test_y)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

168/168 [==============================] - 0s 1ms/step - loss: 0.2211 - accuracy: 0.9477
Test Loss: 0.2211216390132904
Test Accuracy: 0.9476820230484009
